# Automated Essay Scoring 

### What is this Notebook about?

- This Notebook contains all raw results and experiments done with the various deep learning methods. The output cells contains the results that we got. The whole Notebook may take approximately 5-6 hours. This Notebook contains the following:

* Preprocess data
* Important util methods to be used later
* Deep Learning Model declaration
* Running bert for individual sets
* Running bert for whole dataset
* Running word2vec for individual sets
* Running word2vec for whole dataset

# Install and import packages

In [1]:
import tensorflow as tf
import sys
print(tf.__version__)
print(tf.config.list_physical_devices())
print('Hi mom')

2.8.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Hi mom


In [2]:
# !{sys.executable} -m pip install sacremoses

In [3]:
import transformers as ppb

In [4]:
# import important libraries and download data
import os
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import multiprocessing
%matplotlib notebook
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import collections
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')  
# ! git clone https://github.com/Gaurav-Pande/AES_DL.git && mv AES_DL/data .

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/SonTungKieu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/SonTungKieu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/SonTungKieu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Preprocess

In [5]:
# Declaring some visualization methods to plot accuracy and model diagram
def plot_accuracy_curve(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['mse'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

def plot_acrchitecture(filename, model):
    plot_model(model, to_file=str(filename) + '.png')

In [6]:
# method to split data into sets
def split_in_sets(data):
    essay_sets = []
    min_scores = []
    max_scores = []
    
    for s in range(1,9):
        essay_set = data[data["essay_set"] == s]
        essay_set.dropna(axis=1, inplace=True)
        n, d = essay_set.shape
        set_scores = essay_set["domain1_score"]
        print ("Set", s, ": Essays = ", n , "\t Attributes = ", d)
        min_scores.append(set_scores.min())
        max_scores.append(set_scores.max())
        essay_sets.append(essay_set)
    
    return (essay_sets, min_scores, max_scores)

In the below cell, we can see the data we need to operate. We essentially drops the column, we dont need and keep the domain_score only along with essay text.

In [7]:
dataset_path = "./data/training_set_rel3.tsv"
data = pd.read_csv(dataset_path, sep="\t", encoding="ISO-8859-1")
min_scores = [2, 1, 0, 0, 0, 0, 0, 0]
max_scores = [12, 6, 3, 3, 4, 4, 30, 60]
essay_sets, data_min_scores, data_max_scores = split_in_sets(data)
set1, set2, set3, set4, set5, set6, set7, set8 = tuple(essay_sets)
data.dropna(axis=1, inplace=True)
data.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set1.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set2.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set3.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set4.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set5.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set6.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set7.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
set8.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
sets = [set1,set2,set3,set4,set5,set6,set7,set8]
data.head()

/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_18021/2675578851.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  essay_set.dropna(axis=1, inplace=True)
/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_18021/810880094.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set1.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_18021/810880094.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

Set 1 : Essays =  1783 	 Attributes =  6
Set 2 : Essays =  1800 	 Attributes =  9
Set 3 : Essays =  1726 	 Attributes =  6
Set 4 : Essays =  1770 	 Attributes =  6
Set 5 : Essays =  1805 	 Attributes =  6
Set 6 : Essays =  1800 	 Attributes =  6
Set 7 : Essays =  1569 	 Attributes =  14
Set 8 : Essays =  723 	 Attributes =  18


/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_18021/810880094.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set6.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_18021/810880094.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set7.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
/var/folders/pj/ldkmdmbj5bld43hhctfjwh280000gn/T/ipykernel_18021/810880094.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [8]:
data.shape

(12976, 4)

 There are named entity tags as you can see above, which can impact our model. so we need to remove them from the dataframe. So we will extend our stopwords set later on. But first we need to create all possible ner tags.

In [9]:
cap = ['@CAPS'+str(i) for i in range(100)]
loc = ['@LOCATION'+str(i) for i in range(100)]
org =['@ORGANIZATION'+str(i) for i in range(100)]
per = ['@PERSON'+str(i) for i in range(100)]
date = ['@DATE'+str(i) for i in range(100)]
time = ['@TIME'+str(i) for i in range(100)]
money = ['@MONEY'+str(i) for i in range(100)]
ner =  cap + loc + org + per + date + time + money

In [13]:
topic_number = 0
fig, ax = plt.subplots(4,2, figsize=(7, 7), sharey=False)

for i in range(4):
    for j in range(2):
        topic_number += 1
        data[data['essay_set'] == topic_number].groupby('domain1_score')['essay_id'].agg('count').plot.bar(ax=ax[i, j], rot=0, color="maroon")
        ax[i,j].set_title('Essay %i' % topic_number)
        
ax[3,0].locator_params(nbins=10)
ax[3,1].locator_params(nbins=10)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

<IPython.core.display.Javascript object>

# Model

In [15]:
from keras.layers import Embedding, Input, LSTM, Dense, Dropout, Lambda, Flatten, Bidirectional, Conv2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization
from keras.models import Sequential, Model, load_model, model_from_config
import keras.backend as K
import time
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.callbacks import ModelCheckpoint

In [16]:
class BERTung:
    def __init__(self, data):
        self.data = data

    """Define the model."""
    def get_model(self, bidirectional_status:bool = True, Hidden_dim1: int =400, Hidden_dim2:int=128, return_sequence:bool=True, drop_out_value:float=0.5, recurrent_drop_out:float=0.4, inputsize:int=768, activation_func='relu'):
        '''
        bidirectional: False => LSTM,
                       True => Bidirectional LSTM
        '''

        model = Sequential()
        if bidirectional_status:
            model.add(Bidirectional(LSTM(Hidden_dim1, return_sequences=return_sequence , dropout=drop_out_value, recurrent_dropout=recurrent_drop_out), input_shape=[1, inputsize]))
            model.add(Bidirectional(LSTM(Hidden_dim2, recurrent_dropout=recurrent_drop_out)))
        else:
            model.add(LSTM(Hidden_dim1, dropout=drop_out_value, recurrent_dropout=recurrent_drop_out, input_shape=[1, inputsize], return_sequences=return_sequence))
            model.add(LSTM(Hidden_dim2, recurrent_dropout=recurrent_drop_out))

        model.add(Dropout(drop_out_value))
        model.add(Dense(1, activation=activation_func))
        model.add(BatchNormalization())

        model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae', 'mse'])
        return model

    def get_features(self, X, model):
        max_len = 0
        for i in X.values:
            if len(i) > max_len:
                max_len = len(i)

        padded_X = np.array([i + [0]*(max_len-len(i)) for i in X.values])
        attention_mask_X = np.where(padded_X != 0, 1, 0)
        X_input_ids = torch.tensor(padded_X)
        X_attention_mask = torch.tensor(attention_mask_X)

        with torch.no_grad():
            last_hidden_states_X = model(X_input_ids, attention_mask=X_attention_mask)

        X_features = last_hidden_states_X[0][:,0,:].numpy()

        return X_features

    # '''Train model'''
    def train_model(self, model_name, bidirectional=False):
        '''
        BERT: True => BERT
              False => DistilBERT
        '''

        global tokenizer_class
        cv = KFold(n_splits=5, shuffle=True)
        X = self.data
        y = self.data['domain1_score']
        cv_data = cv.split(X)

        results = []
        prediction_list = []
        fold_count = 1

        if model_name == 'BERT':
            model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
        elif model_name == 'DistilBERT':
            model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
        elif model_name == 'ALBERT':
            model_class, tokenizer_class, pretrained_weights = (ppb.AlbertModel, ppb.AlbertTokenizer, 'albert-large-v2')

        # Load pretrained model/tokenizer
        tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
        model = model_class.from_pretrained(pretrained_weights)

        for traincv, testcv in cv_data:
            print("\n--------Fold {}--------\n".format(fold_count))

            # get the train and test from the dataset.
            X_train, X_test, y_train, y_test = X.iloc[traincv], X.iloc[testcv], y.iloc[traincv], y.iloc[testcv]
            train_essays = X_train['essay']
            test_essays = X_test['essay']
        #     print(train_essays)

            sentences = []
            tokenize_sentences = []
            train_bert_embeddings = []

            tokenized_train = train_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=55)))
            tokenized_test = test_essays.apply((lambda x: tokenizer.encode(x, add_special_tokens=True ,max_length=55)))


            train_features = self.get_features(tokenized_train, model)
            test_features = self.get_features(tokenized_test, model)

            print(train_features[:1])
            print(test_features[:1])
    #
    #         train_x, train_y = train_features.shape
    #         test_x,test_y = test_features.shape
    #         trainDataVectors = np.reshape(train_features, (train_x, 1, train_y))
    #         testDataVectors = np.reshape(test_features, (test_x, 1, test_y))
    #
    #         lstm_model = self.get_model(Hidden_dim1=Hidden_dim1, Hidden_dim2=Hidden_dim2, return_sequences=return_sequences,
    #                                 dropout=dropout, recurrent_dropout=recurrent_dropout, input_size=input_size,
    #                                 activation=activation, bidirectional=bidirectional)
    #         history = lstm_model.fit(trainDataVectors, y_train, batch_size=batch_size, epochs=epoch)
    #
    #         plot_accuracy_curve(history)
    #         y_pred = lstm_model.predict(testDataVectors)
    #         y_pred = np.around(y_pred)
    #         #y_pred.dropna()
    #         np.nan_to_num(y_pred)
    #
    #         # evaluate the model
    #         print(y_pred)
    #         result = cohen_kappa_score(y_test.values, y_pred, weights='quadratic')
    #         print("Kappa Score: {}".format(result))
    #         results.append(result)
    #         fold_count +=1
    #
    #         tf.keras.backend.clear_session()
    #
    #     print("Average kappa score value is : {}".format(np.mean(np.asarray(results))))
    #
    #     return results
    #



In [72]:
# !{sys.executable} -m pip install sentencepiece

In [18]:
model = BERTung(data)
model.train_model('DistilBERT')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the 


--------Fold 1--------



KeyboardInterrupt: 

In [0]:
import tensorflow as tf
tf.keras.backend.clear_session()

Below method is for running the model on whole dataset using the word2vec model.